In [ ]:
import os
import time
import sys
import json
import numpy as np
import torch as th
import matplotlib.pyplot as plt
import motornet as mn
from simple_policy import Policy
from simple_task import CentreOutFFMinJerk
from simple_utils import *
from tqdm import tqdm

print('All packages imported.')
print('pytorch version: ' + th.__version__)
print('numpy version: ' + np.__version__)
print('motornet version: ' + mn.__version__)

In [ ]:
device = th.device("cpu")

effector = mn.effector.RigidTendonArm26(muscle=mn.muscle.RigidTendonHillMuscle())
env = CentreOutFFMinJerk(effector=effector, max_ep_duration=1.)

policy = Policy(env.observation_space.shape[0], 128, env.n_muscles, device=device)
optimizer = th.optim.Adam(policy.parameters(), lr=10**-3)

In [ ]:
batch_size =  512
n_batch    =  30000

losses = {
    'overall': [],
    'position': [],
    'angle': [],
    'lateral': [],
    'muscle': [],
    'hidden': []}

for batch in tqdm(range(n_batch),
                  desc=f"Training {n_batch} batches of {batch_size}",
                  unit="batch"):

    data = run_episode(env, policy, batch_size, catch_trial_perc=50, condition='train', ff_coefficient=0.0, detach=False)
    loss, _, muscle_loss, hidden_loss, _, _ = cal_loss(data, env.muscle.max_iso_force, env.dt, policy, test=False)

    # backward pass & update weights
    optimizer.zero_grad() 
    loss.backward()
    th.nn.utils.clip_grad_norm_(policy.parameters(), max_norm=1.)  # important!
    optimizer.step()

    # TEST
    # Run episode
    data = run_episode(env,policy,8,0,'test',ff_coefficient=0.0,detach=True)

    # calculate losses
    _, position_loss, _, _, angle_loss, lateral_loss = cal_loss(data, env.muscle.max_iso_force, env.dt, policy, test=True)

    # Update loss values in the dictionary
    losses['overall'].append(loss.item())
    losses['position'].append(position_loss.item())
    losses['angle'].append(angle_loss.item())
    losses['lateral'].append(lateral_loss.item())
    losses['muscle'].append(muscle_loss.item())
    losses['hidden'].append(hidden_loss.item())



In [ ]:
weight_file = 'simple_weights'
log_file    = 'simple_log.json'
cfg_file    = 'simple_cfg.json'

# save model weights
th.save(policy.state_dict(), weight_file)

# save training history (log)
with open(log_file, 'w') as file:
    json.dump({'losses':losses}, file)

# save environment configuration dictionary
cfg = env.get_save_config()
with open(cfg_file, 'w') as file:
    json.dump(cfg, file)

In [ ]:
log = json.load(open(log_file,'r'))
fig,ax = plot_training_log(log=log["losses"],loss_type='position', w=1)

In [ ]:
# TEST NETWORK ON CENTRE-OUT

data = test(cfg_file, weight_file)

fig, ax = plot_simulations(xy=data['xy'], target_xy=data['tg'], figsize=(8,6))
fig, ax = plot_activation(data['all_hidden'], data['all_muscle'])

In [ ]:
fig, ax = plot_kinematics(all_xy=data["xy"], all_tg=data["tg"], all_vel=data["vel"])